# Ivy Bert Demo
--------------------------------------------------------------------------------

In this demo, we show how a [Bidirectional Encoder From Transformers (Bert)](https://pytorch.org/hub/huggingface_pytorch-transformers/) model written in Ivy native code  used for **Sequence Classification** and **MLM**, and integrated with all three of the major ML frameworks: **PyTorch**, **TensorFlow** and **JAX**.

**First of all**
You first have to enable gpu support if you are in **Google Colab**

Go to **Runtime** -> **Change runtime type** -> **Choose Gpu**

## Install the dependecies

- ivy `ivy library`
- haiku `Haiku framework for jax`
- ivy_models `ivy models library`
- transformers ` Transformers library to get the pretrained model`

**If you have all of the libraries installed you can save some time and skip this cell if not you should run this cell and restart the notebook**

In [ ]:
!pip install -q ivy
!pip install -q dm-haiku

!pip install git+https://github.com/mohame54/models.git
!pip install transformers


## Import the modules

In [2]:
import torch
import ivy
import ivy_models
from transformers import AutoModel, AutoTokenizer
import warnings
import numpy as np
warnings.filterwarnings("ignore") # to ignore the warnings

## Data Preparation


**load the pretrained Model and tokenizer**




In [ ]:
bert_base = AutoModel.from_pretrained("bert-base-uncased")
bert_base = bert_base.eval() # for inference and evaluation
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [4]:
# Prepare some samples to test on

texts = ["i did't really like your tone."]
inputs = bert_tokenizer(texts,
                        padding='longest',
                        return_tensors='pt',
                        max_length=512)
inputs

{'input_ids': tensor([[ 101, 1045, 2106, 1005, 1056, 2428, 2066, 2115, 4309, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

**We get the transformers Bert pooler outputs to compare it with ivy bert outputs**

In [5]:
# torch model inference
with torch.no_grad():
   bert_output = bert_base(**inputs).pooler_output

##Ivy model Inference with numpy

**First We import [the native ivy code for Bert](https://github.com/unifyai/models/blob/master/ivy_models/bert/bert.py)**

In [ ]:
ivy.set_backend('numpy')
ivy_bert = ivy_models.bert_base_uncased(pretrained=True)

In [ ]:
ivy_inputs = {k:ivy.asarray(v.numpy()) for k, v in inputs.items()}
ivy_bert.compile(kwargs=ivy_inputs)

### Ivy inference with Sequence Classification

In [ ]:
import numpy as np
ivy_output = ivy_bert(**ivy_inputs)['pooler_output']

In [ ]:
print(f"logits shapes, Ivy: {list(ivy_output.shape)}, torch: {list(bert_output.shape)}")
logits_close = np.allclose(ivy_output, bert_output.detach().numpy(),rtol=0.005,atol=0.005)
if logits_close:
  print(f"logits are equal")
else:
  print(f"logits are not equal")

logits shapes, Ivy: [1, 768], torch: [1, 768]
logits are equal


## Ivy model inference with tensorflow

In [ ]:
ivy.set_backend('tensorflow')
ivy_bert = ivy_models.bert_base_uncased(pretrained=True)

**Let's compare the runtime before and after compilation**

In [ ]:
import time

st = time.time()
ivy_inputs = {k:ivy.asarray(v.numpy()) for k, v in inputs.items()}
ivy_output = ivy_bert(**ivy_inputs)['pooler_output']
fn = time.time()
print(f"Finished in {(fn - st):.2f} secs")

Finished in 89.43 secs


In [ ]:
print(f"logits shapes, Ivy: {list(ivy_output.shape)}, torch: {list(bert_output.shape)}")
logits_close = np.allclose(ivy_output.numpy(), bert_output.detach().numpy(),rtol=0.005,atol=0.005)
if logits_close:
  print(f"logits are equal")
else:
  print(f"logits are not equal")

logits shapes, Ivy: [1, 768], torch: [1, 768]
logits are equal


**Now we compile the model**

We repeat the same procedure before

In [ ]:
ivy_bert.compile(kwargs=ivy_inputs)

In [ ]:
st = time.time()
ivy_output = ivy_bert(**ivy_inputs)['pooler_output']
fn = time.time()
print(f"Finished in {(fn - st):.2f} secs")

Finished in 0.60 secs


**We can see that the big difference in inference runtime before and after compilation**

In [ ]:
print(f"logits shapes, Ivy: {list(ivy_output.shape)}, torch: {list(bert_output.shape)}")
logits_close = np.allclose(ivy_output.numpy(), bert_output.detach().numpy(),rtol=0.005,atol=0.005)
if logits_close:
  print(f"logits are equal")
else:
  print(f"logits are not equal")

logits shapes, Ivy: [1, 768], torch: [1, 768]
logits are equal


## Ivy model inference with Jax

In [ ]:
import jax
import jax.numpy as jnp
jax.config.update('jax_enable_x64', True)
ivy.set_backend("jax")
ivy_bert = ivy_models.bert_base_uncased(pretrained=True)

In [ ]:
ivy_inputs = {k:ivy.asarray(v.numpy()) for k, v in inputs.items()}
ivy_bert.compile(kwargs=ivy_inputs)

In [ ]:
ivy_output = ivy_bert(**ivy_inputs)['pooler_output']
print(f"logits shapes, Ivy: {list(ivy_output.shape)}, torch: {list(bert_output.shape)}")
ref = jnp.array( bert_output.detach())
logits_close = jnp.allclose(ivy_output, ref,rtol=0.005,atol=0.005)
if logits_close:
  print(f"logits are equal")
else:
  print(f"logits are not equal")

logits shapes, Ivy: [1, 768], torch: [1, 768]
logits are equal


## Ivy model inference with torch

**Initialize the model also compile it for fast inference**

In [ ]:
ivy.set_backend("torch")
ivy_bert = ivy_models.bert_base_uncased(pretrained=True)
ivy_inputs = {k:ivy.asarray(v.numpy()) for k, v in inputs.items()}
ivy_bert.compile(kwargs=ivy_inputs)

**Check logits values and the shapes of logits as before**

In [9]:
ivy_output = ivy_bert(**ivy_inputs)['pooler_output']
print(f"logits shapes, Ivy: {list(ivy_output.shape)}, torch: {list(bert_output.shape)}")
ref = bert_output.detach()
logits_close = torch.allclose(ivy_output, ref.cuda(),rtol=0.005,atol=0.005)
if logits_close:
  print(f"logits are equal")
else:
  print(f"logits are not equal")

logits shapes, Ivy: [1, 768], torch: [1, 768]
logits are equal
